In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pprint import pprint
import traceback
import urllib

# Define the path to the Delta table
delta_table_path = "dbfs:/user/hive/warehouse/authentication_credentials"

# Read the delta table to a Spark DataFrame
aws_keys_df = spark.read.format('delta').load(delta_table_path)

ACCESS_KEY = aws_keys_df.select('Access Key ID').collect()[0]['Access Key ID']
SECRET_KEY = aws_keys_df.select('Secret access key').collect()[0]['Secret access key']

ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")

# Encode the secrete key
ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY.encode('utf-8'), safe="")

# AWS S3 bucket name
AWS_S3_BUCKET = "user-0a2f66c3e41f-bucket"
# Mount name for the bucket
# MOUNT_NAME = "/mnt/0a2f66c3e41f.df_geo"
MOUNT_NAMES = {
                "/mnt/0a2f66c3e41f.df_pin" : "0a2f66c3e41f.pin", 
                "/mnt/0a2f66c3e41f.df_geo" : "0a2f66c3e41f.geo", 
                "/mnt/0a2f66c3e41f.df_user" : "0a2f66c3e41f.user"
                }
# BUCKETS = ["0a2f66c3e41f.pin", "0a2f66c3e41f.geo", "0a2f66c3e41f.user"]
# Source url
SOURCE_URL = "s3n://{0}:{1}@{2}".format(ACCESS_KEY, ENCODED_SECRET_KEY, AWS_S3_BUCKET)

for MOUNT_NAME, BUCKET in MOUNT_NAMES.items():
    # Check if mount already exists to avoid error
    if MOUNT_NAME not in [mnt.mountPoint for mnt in dbutils.fs.mounts()]:
        # Mount the drive
        dbutils.fs.mount(SOURCE_URL, MOUNT_NAME)
    else:
        print(f"Mount '{MOUNT_NAME}' already exists")
        #  Unmount the directory if it's already mounted
        dbutils.fs.unmount(MOUNT_NAME)

        # Mount the drive again
        dbutils.fs.mount(SOURCE_URL, MOUNT_NAME)

        # pprint([mnt.mountPoint for mnt in dbutils.fs.mounts()])
        # # Check the content of mounted S3 bucket
        # display(dbutils.fs.ls("/mnt/0a2f66c3e41f.df_geo"))

        # Read data from the mounted S3 bucket
        try:
            display(dbutils.fs.ls(f"/mnt/{MOUNT_NAME}/topics/{BUCKET}/partition=0/"))
            print('SUCCESS')
        except Exception as e:
            print("FAILED")
            # Uncomment the line below to print the traceback
            traceback.print_exc()

        # # File location and type
        file_location = f"{MOUNT_NAME}/topics/{BUCKET}/partition=0/*.json"
        file_type = "json"

        # # Ask Spark to infer the schema
        infer_schema = "true"

        # Read in JSONs from mounted S3 bucket
        df = spark.read.format(file_type).option("inferSchema", infer_schema).load(file_location)

        # Display Spark dataframe to check its content
        display(df)

+--------------------+--------------------+
Mount '/mnt/0a2f66c3e41f.df_pin' already exists
/mnt/0a2f66c3e41f.df_pin has been unmounted.
FAILED
Traceback (most recent call last):
 File "<command-198588058359756>", line 56, in <module>
 display(dbutils.fs.ls(f"/mnt/{MOUNT_NAME}/topics/{BUCKET}/partition=0/"))
 File "/databricks/python_shell/dbruntime/dbutils.py", line 389, in f_with_exception_handling
 raise exc
dbruntime.dbutils.DBUtils.FSHandler.prettify_exception_message.<locals>.f_with_exception_handling.<locals>.ExecutionError: An error occurred while calling z:com.databricks.backend.daemon.dbutils.FSUtils.ls.
: java.io.FileNotFoundException: No such file or directory: s3n://user-12c0d092d679-bucket/mnt/0a2f66c3e41f.df_pin/topics/0a2f66c3e41f.pin/partition=0
	at shaded.databricks.org.apache.hadoop.fs.s3a.S3AFileSystem.s3GetFileStatus(S3AFileSystem.java:3390)
	at shaded.databricks.org.apache.hadoop.fs.s3a.S3AFileSystem.innerGetFileStatus(S3AFileSystem.java:3264)
	at shaded.databricks.org.apache.hadoop.fs.s3a.S3AFileSystem.getFileStatus(S3AFileSystem.java:3203)
	at shaded.databricks.org.apache.hadoop.fs.s3a.S3AFileSystem.innerListStatus(S3AFileSystem.java:2726)
	at shaded.databricks.org.apache.hadoop.fs.s3a.S3AFileSystem.lambda$listStatus$11(S3AFileSystem.java:2705)
	at shaded.databricks.org.apache.hadoop.fs.s3a.Invoker.once(Invoker.java:118)
	at shaded.databricks.org.apache.hadoop.fs.s3a.Invoker.once(Invoker.java:112)
	at shaded.databricks.org.apache.hadoop.fs.s3a.S3AFileSystem.listStatus(S3AFileSystem.java:2705)
	at com.databricks.backend.daemon.data.client.DBFSV2.$anonfun$listStatus$2(DatabricksFileSystemV2.scala:95)
	at com.databricks.s3a.S3AExceptionUtils$.convertAWSExceptionToJavaIOException(DatabricksStreamUtils.scala:70)
	at com.databricks.backend.daemon.data.client.DBFSV2.$anonfun$listStatus$1(DatabricksFileSystemV2.scala:92)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$1(UsageLogging.scala:330)
	at com.databricks.logging.UsageLogging.executeThunkAndCaptureResultTags$1(UsageLogging.scala:424)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperationWithResultTags$4(UsageLogging.scala:444)
	at com.databricks.logging.Log4jUsageLoggingShim$.$anonfun$withAttributionContext$1(Log4jUsageLoggingShim.scala:33)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(AttributionContext.scala:94)
	at com.databricks.logging.Log4jUsageLoggingShim$.withAttributionContext(Log4jUsageLoggingShim.scala:31)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:205)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:204)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.withAttributionContext(DatabricksFileSystemV2.scala:510)
	at com.databricks.logging.UsageLogging.withAttributionTags(UsageLogging.scala:240)
	at com.databricks.logging.UsageLogging.withAttributionTags$(UsageLogging.scala:225)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.withAttributionTags(DatabricksFileSystemV2.scala:510)
	at com.databricks.logging.UsageLogging.recordOperationWithResultTags(UsageLogging.scala:419)
	at com.databricks.logging.UsageLogging.recordOperationWithResultTags$(UsageLogging.scala:339)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.recordOperationWithResultTags(DatabricksFileSystemV2.scala:510)
	at com.databricks.logging.UsageLogging.recordOperation(UsageLogging.scala:330)
	at com.databricks.logging.UsageLogging.recordOperation$(UsageLogging.scala:302)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.recordOperation(DatabricksFileSystemV2.scala:510)
	at com.databricks.backend.daemon.data.client.DBFSV2.listStatus(DatabricksFileSystemV2.scala:92)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystem.listStatus(DatabricksFileSystem.scala:164)
	at com.databricks.backend.daemon.dbutils.FSUtils$.$anonfun$

category,downloaded,follower_count,image_src,index,is_image_or_video,poster_name,save_location,tag_list,title,unique_id
christmas,1,46k,https://i.pinimg.com/videos/thumbnails/originals/40/83/f5/4083f5b4971bf235f89a4784ab87271e.0000001.jpg,2482,video,"Life on Summerhill | Home, Holiday Decor & DIY Website",Local save in /data/christmas,"Diy Christmas Decorations For Home,Farmhouse Christmas Decor,Christmas Home,Christmas Holidays,Christmas Front Porches,How To Decorate For Christmas,Christmas Porch Ideas,Christmas Decorating Ideas,Large Outdoor Christmas Decorations",FORNT PORCH CHRISTMAS DECORATING IDEAS,08604f20-fa17-4b9a-9949-781717eca6cd
education,1,5k,https://i.pinimg.com/originals/4b/b9/a8/4bb9a85f609a8384ed9c9ab14e7bcea0.jpg,4137,image,"Write Your Story | Budgeting, Healthy Lifestyle, Faith",Local save in /data/education,"1st Year Teachers,First Year Teaching,First Grade Teachers,Elementary Teacher,Montessori Elementary,Teaching Reading,History Teachers,High School Classroom,First Grade Classroom",15 Essential Classroom Items For First Year Teachers / First Year Teacher Must Haves,bf275054-4464-448b-9143-1bf101aa366d
quotes,1,45k,https://i.pinimg.com/originals/ee/b4/bc/eeb4bccdb074edb787ea61711e0c4dfd.jpg,7768,image,"Boss Babe Chronicles | Personal Development, Finances, Self Care",Local save in /data/quotes,"Motivacional Quotes,Motivational Quotes For Life,Uplifting Quotes,True Quotes,Words Quotes,Empowering Quotes,Boss Up Quotes,Inspirational Success Quotes,Boss Babe Quotes Work Hard",16 Motivational Quotes Get You Inspired Today - Boss Babe Chronicles,ae1a61f6-d958-40fe-ba93-6a78b3a73178
travel,1,10k,https://i.pinimg.com/originals/30/93/cb/3093cb01d9de2d125fda8ba5e3e41946.jpg,10138,image,"Wanderlust Chloe ✈️ Travel guides, inspo and adventure travel ✈️",Local save in /data/travel,"Costa Rica Travel,Rio Celeste Costa Rica,Dream Vacations,Vacation Spots,Vacation Travel,Travel Pictures,Travel Photos,Fortuna Costa Rica,Costa Rica Pictures","14 Amazing Things To Do In Costa Rica | Volcanoes, Waterfalls, Wildlife And More",927c4658-cc3f-4b92-9b5c-70743d0c238d
christmas,1,500k,https://i.pinimg.com/originals/aa/6d/0f/aa6d0f44d7c1c96b998cb9aa6c4446b8.png,2418,image,HikenDip,Local save in /data/christmas,"Farmhouse Christmas Decor,Rustic Christmas,Christmas Time,Vintage Christmas,Xmas,Primitive Christmas Crafts,Christmas Vignette,Indoor Christmas Decorations,Diy Christmas Ornaments",100 DIY Christmas Centerpieces You'll Love To Decorate Your Home With For The Christmas Season - Hike n Dip,da8745a6-5160-46c4-877d-181d50a729fd
christmas,1,500k,https://i.pinimg.com/originals/aa/6d/0f/aa6d0f44d7c1c96b998cb9aa6c4446b8.png,2418,image,HikenDip,Local save in /data/christmas,"Farmhouse Christmas Decor,Rustic Christmas,Christmas Time,Vintage Christmas,Xmas,Primitive Christmas Crafts,Christmas Vignette,Indoor Christmas Decorations,Diy Christmas Ornaments",100 DIY Christmas Centerpieces You'll Love To Decorate Your Home With For The Christmas Season - Hike n Dip,da8745a6-5160-46c4-877d-181d50a729fd
diy-and-crafts,1,892k,https://i.pinimg.com/originals/ff/fe/38/fffe384f3ec18a0d87cb2d80cc8c1499.jpg,3156,image,Michelle {CraftyMorning.com},Local save in /data/diy-and-crafts,"Christmas Gifts For Parents,Christmas Decorations For Kids,Christmas Crafts For Toddlers,Preschool Christmas,Christmas Crafts For Gifts,Christmas Activities,Toddler Crafts,Kids Christmas,Christmas Feeling",Handprint Reindeer Ornaments - Crafty Morning,fa6e31a4-18c2-4eca-a6d8-e903eee2c2a4
diy-and-crafts,1,892k,https://i.pinimg.com/originals/ff/fe/38/fffe384f3ec18a0d87cb2d80cc8c1499.jpg,3156,image,Michelle {CraftyMorning.com},Local save in /data/diy-and-crafts,"Christmas Gifts For Parents,Christmas Decorations For Kids,Christmas Crafts For Toddlers,Preschool Christmas,Christmas Crafts For Gifts,Christmas Activities,Toddler Crafts,Kids Christmas,Christmas Feeling",Handprint Reindeer Ornaments - Crafty Morning,fa6e31a4-18c2-4eca-a6d8-e903eee2c2a4
home-decor,1,92k,https://i.pini

Mount '/mnt/0a2f66c3e41f.df_geo' already exists
/mnt/0a2f66c3e41f.df_geo has been unmounted.
FAILED
Traceback (most recent call last):
 File "<command-198588058359756>", line 56, in <module>
 display(dbutils.fs.ls(f"/mnt/{MOUNT_NAME}/topics/{BUCKET}/partition=0/"))
 File "/databricks/python_shell/dbruntime/dbutils.py", line 389, in f_with_exception_handling
 raise exc
dbruntime.dbutils.DBUtils.FSHandler.prettify_exception_message.<locals>.f_with_exception_handling.<locals>.ExecutionError: An error occurred while calling z:com.databricks.backend.daemon.dbutils.FSUtils.ls.
: java.io.FileNotFoundException: No such file or directory: s3n://user-12c0d092d679-bucket/mnt/0a2f66c3e41f.df_geo/topics/0a2f66c3e41f.geo/partition=0
	at shaded.databricks.org.apache.hadoop.fs.s3a.S3AFileSystem.s3GetFileStatus(S3AFileSystem.java:3390)
	at shaded.databricks.org.apache.hadoop.fs.s3a.S3AFileSystem.innerGetFileStatus(S3AFileSystem.java:3264)
	at shaded.databricks.org.apache.hadoop.fs.s3a.S3AFileSystem.getFileStatus(S3AFileSystem.java:3203)
	at shaded.databricks.org.apache.hadoop.fs.s3a.S3AFileSystem.innerListStatus(S3AFileSystem.java:2726)
	at shaded.databricks.org.apache.hadoop.fs.s3a.S3AFileSystem.lambda$listStatus$11(S3AFileSystem.java:2705)
	at shaded.databricks.org.apache.hadoop.fs.s3a.Invoker.once(Invoker.java:118)
	at shaded.databricks.org.apache.hadoop.fs.s3a.Invoker.once(Invoker.java:112)
	at shaded.databricks.org.apache.hadoop.fs.s3a.S3AFileSystem.listStatus(S3AFileSystem.java:2705)
	at com.databricks.backend.daemon.data.client.DBFSV2.$anonfun$listStatus$2(DatabricksFileSystemV2.scala:95)
	at com.databricks.s3a.S3AExceptionUtils$.convertAWSExceptionToJavaIOException(DatabricksStreamUtils.scala:70)
	at com.databricks.backend.daemon.data.client.DBFSV2.$anonfun$listStatus$1(DatabricksFileSystemV2.scala:92)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$1(UsageLogging.scala:330)
	at com.databricks.logging.UsageLogging.executeThunkAndCaptureResultTags$1(UsageLogging.scala:424)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperationWithResultTags$4(UsageLogging.scala:444)
	at com.databricks.logging.Log4jUsageLoggingShim$.$anonfun$withAttributionContext$1(Log4jUsageLoggingShim.scala:33)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(AttributionContext.scala:94)
	at com.databricks.logging.Log4jUsageLoggingShim$.withAttributionContext(Log4jUsageLoggingShim.scala:31)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:205)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:204)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.withAttributionContext(DatabricksFileSystemV2.scala:510)
	at com.databricks.logging.UsageLogging.withAttributionTags(UsageLogging.scala:240)
	at com.databricks.logging.UsageLogging.withAttributionTags$(UsageLogging.scala:225)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.withAttributionTags(DatabricksFileSystemV2.scala:510)
	at com.databricks.logging.UsageLogging.recordOperationWithResultTags(UsageLogging.scala:419)
	at com.databricks.logging.UsageLogging.recordOperationWithResultTags$(UsageLogging.scala:339)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.recordOperationWithResultTags(DatabricksFileSystemV2.scala:510)
	at com.databricks.logging.UsageLogging.recordOperation(UsageLogging.scala:330)
	at com.databricks.logging.UsageLogging.recordOperation$(UsageLogging.scala:302)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.recordOperation(DatabricksFileSystemV2.scala:510)
	at com.databricks.backend.daemon.data.client.DBFSV2.listStatus(DatabricksFileSystemV2.scala:92)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystem.listStatus(DatabricksFileSystem.scala:164)
	at com.databricks.backend.daemon.dbutils.FSUtils$.$anonfun$ls$1(DBUtilsCore.scala:157)
	at com.databric

country,index,latitude,longitude,timestamp
Antarctica (the territory South of 60 deg S),2418,-88.4642,-171.061,2022-05-27T11:30:59
Antarctica (the territory South of 60 deg S),2418,-88.4642,-171.061,2022-05-27T11:30:59
Cocos (Keeling) Islands,10794,-89.5236,-154.567,2022-01-01T02:26:50
Cocos (Keeling) Islands,10794,-89.5236,-154.567,2022-01-01T02:26:50
Central African Republic,2074,-52.3213,-50.11,2019-11-03T05:41:59
Central African Republic,2074,-52.3213,-50.11,2019-11-03T05:41:59
British Virgin Islands,2293,-87.7946,-159.647,2022-03-21T10:46:53
Saint Kitts and Nevis,10663,-27.3474,-162.83,2019-07-25T18:53:51
Antigua and Barbuda,7922,-88.0974,-172.052,2021-01-27T09:14:19
Antigua and Barbuda,8606,-88.0974,-172.052,2021-03-28T14:54:07


Mount '/mnt/0a2f66c3e41f.df_user' already exists
/mnt/0a2f66c3e41f.df_user has been unmounted.
FAILED
Traceback (most recent call last):
 File "<command-198588058359756>", line 56, in <module>
 display(dbutils.fs.ls(f"/mnt/{MOUNT_NAME}/topics/{BUCKET}/partition=0/"))
 File "/databricks/python_shell/dbruntime/dbutils.py", line 389, in f_with_exception_handling
 raise exc
dbruntime.dbutils.DBUtils.FSHandler.prettify_exception_message.<locals>.f_with_exception_handling.<locals>.ExecutionError: An error occurred while calling z:com.databricks.backend.daemon.dbutils.FSUtils.ls.
: java.io.FileNotFoundException: No such file or directory: s3n://user-12c0d092d679-bucket/mnt/0a2f66c3e41f.df_user/topics/0a2f66c3e41f.user/partition=0
	at shaded.databricks.org.apache.hadoop.fs.s3a.S3AFileSystem.s3GetFileStatus(S3AFileSystem.java:3390)
	at shaded.databricks.org.apache.hadoop.fs.s3a.S3AFileSystem.innerGetFileStatus(S3AFileSystem.java:3264)
	at shaded.databricks.org.apache.hadoop.fs.s3a.S3AFileSystem.getFileStatus(S3AFileSystem.java:3203)
	at shaded.databricks.org.apache.hadoop.fs.s3a.S3AFileSystem.innerListStatus(S3AFileSystem.java:2726)
	at shaded.databricks.org.apache.hadoop.fs.s3a.S3AFileSystem.lambda$listStatus$11(S3AFileSystem.java:2705)
	at shaded.databricks.org.apache.hadoop.fs.s3a.Invoker.once(Invoker.java:118)
	at shaded.databricks.org.apache.hadoop.fs.s3a.Invoker.once(Invoker.java:112)
	at shaded.databricks.org.apache.hadoop.fs.s3a.S3AFileSystem.listStatus(S3AFileSystem.java:2705)
	at com.databricks.backend.daemon.data.client.DBFSV2.$anonfun$listStatus$2(DatabricksFileSystemV2.scala:95)
	at com.databricks.s3a.S3AExceptionUtils$.convertAWSExceptionToJavaIOException(DatabricksStreamUtils.scala:70)
	at com.databricks.backend.daemon.data.client.DBFSV2.$anonfun$listStatus$1(DatabricksFileSystemV2.scala:92)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$1(UsageLogging.scala:330)
	at com.databricks.logging.UsageLogging.executeThunkAndCaptureResultTags$1(UsageLogging.scala:424)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperationWithResultTags$4(UsageLogging.scala:444)
	at com.databricks.logging.Log4jUsageLoggingShim$.$anonfun$withAttributionContext$1(Log4jUsageLoggingShim.scala:33)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(AttributionContext.scala:94)
	at com.databricks.logging.Log4jUsageLoggingShim$.withAttributionContext(Log4jUsageLoggingShim.scala:31)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:205)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:204)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.withAttributionContext(DatabricksFileSystemV2.scala:510)
	at com.databricks.logging.UsageLogging.withAttributionTags(UsageLogging.scala:240)
	at com.databricks.logging.UsageLogging.withAttributionTags$(UsageLogging.scala:225)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.withAttributionTags(DatabricksFileSystemV2.scala:510)
	at com.databricks.logging.UsageLogging.recordOperationWithResultTags(UsageLogging.scala:419)
	at com.databricks.logging.UsageLogging.recordOperationWithResultTags$(UsageLogging.scala:339)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.recordOperationWithResultTags(DatabricksFileSystemV2.scala:510)
	at com.databricks.logging.UsageLogging.recordOperation(UsageLogging.scala:330)
	at com.databricks.logging.UsageLogging.recordOperation$(UsageLogging.scala:302)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.recordOperation(DatabricksFileSystemV2.scala:510)
	at com.databricks.backend.daemon.data.client.DBFSV2.listStatus(DatabricksFileSystemV2.scala:92)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystem.listStatus(DatabricksFileSystem.scala:164)
	at com.databricks.backend.daemon.dbutils.FSUtils$.$anonfun$ls$1(DBUtilsCore.scala:157)
	at com.data

age,date_joined,first_name,index,last_name
27,2016-03-08T13:38:37,Christopher,2015,Bradshaw
20,2015-12-01T15:08:31,Christopher,5076,Butler
39,2017-07-19T07:12:04,Michelle,7790,Gutierrez
49,2016-04-22T20:36:02,Brittany,10509,Thompson
21,2015-11-10T09:27:42,Andrea,8731,Alexander
21,2015-11-10T09:27:42,Andrea,8731,Alexander
24,2016-03-31T20:56:39,Austin,8887,Rodriguez
23,2015-12-01T18:15:02,Christine,7768,Cortez
36,2015-12-20T16:38:13,Michelle,4315,Prince
32,2017-10-10T20:09:33,Christian,10625,Lang
